In [6]:
import pandas as pd
dates=pd.read_csv('data/dates.csv', sep=',')
movies=pd.read_csv('data/movies.csv', sep=',')
ratings1=pd.read_csv('data/ratings1.csv', sep=',')
ratings2=pd.read_csv('data/ratings2.csv', sep=',')
ratings=pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates=pd.concat(
    [ratings, dates],
    axis=1
)
# Объединим таблицы типом left. 
# Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:
joined_false=ratings_dates.join(
    movies,
    rsuffix='_right', # если посмотреть в таблу то можно увидеть что появился солбец "movieid_right" !!!
    how='left'
)
display(joined_false)
# !!!!! 
# Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: 
# один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right).
# Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. 
# Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец 
# в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). 
# Также необходимо указать название ключа в параметре on:
joined=ratings_dates.join(
    movies.set_index('movieId'), #установка ключевого столца из правой таблы, по котороиу будет произведена операция
    on='movieId', # выбран столбец левой таблицы, в который будет перезалит столбец из правого столбца
    how='left' # ну и сам способ контактенации
)
display(joined)
# В результате такого объединения для каждого идентификатора фильма movieId в таблице ratings_dates найден 
# совпадающий с ним идентификатор movieId в таблице movies и присоединена информация 
# о самом фильме (title и genres). Это как раз то, что нам нужно.

# метод Merge:
merged=ratings_dates.merge(
    movies, #присоединяемая таблица
    on='movieId', # выбран столбецпо которому будет слияние
    how='left' # способ контактенации
)
display(merged.head())
# Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:

print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

# ОСОБЕННОСТИ ИСПОЛЬЗОВАНИЯ MERGE() !!!!!!!!!!!!!!!!!!!!!!!!!!!
# Возникает вопрос: почему мы выбрали тип объединения left, а не full, например? 
# Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, 
# но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
display(merged2.tail())
# Результирующее число строк в таблице увеличилось. Но за счёт чего?
# Оказывается, в таблице movies содержались фильмы, которым ещё не были выставлены оценки. 
# В результате объединения типом full outer информация о фильмах перенеслась из таблицы movies в результирующую 
# таблицу. Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие столбцы 
# таблицы ratings_dates заполнились пропусками (NaN). Такие фильмы были записаны в конец таблицы.

# объединение таблиц через Merge c типом Outer:
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)
# !!!!!!!!!!!!!если структура одинаковая , то в итоговой таблице сразу уйдут дубликаты!!!!!!!!!


,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN,NaN,NaN


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True
